In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# P1 Event Time + Replay\n",
        "\n",
        "Uses replay on the same synthetic log to confirm identical results (determinism)."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "\n",
        "from ssp.core.config import SyntheticConfig\n",
        "from ssp.datasets.synthetic import generate_synthetic\n",
        "from ssp.streaming.replay import replay_log\n",
        "from ssp.p1_detect.pipeline import P1Spec, run_p1\n",
        "\n",
        "cfg = SyntheticConfig(seed=4, n_sensors=6, n_steps=12000, dt_s=0.01, anomaly_rate=0.002)\n",
        "ds = generate_synthetic(cfg)\n",
        "\n",
        "log = list(zip(ds.t_event_s.tolist(), ds.y.tolist(), ds.sensor_id.tolist()))\n",
        "log2 = list(replay_log(log))\n",
        "\n",
        "t2 = np.asarray([a for a, _, _ in log2], dtype=np.float64)\n",
        "y2 = np.asarray([b for _, b, _ in log2], dtype=np.float64)\n",
        "s2 = np.asarray([c for _, _, c in log2], dtype=np.int64)\n",
        "\n",
        "p1a = run_p1(ds.y, ds.t_event_s, ds.sensor_id, ds.is_anomaly, P1Spec())\n",
        "p1b = run_p1(y2, t2, s2, ds.is_anomaly, P1Spec())\n",
        "\n",
        "print('scores identical:', np.allclose(p1a['score'], p1b['score']))\n",
        "print('alerts identical:', np.array_equal(p1a['alert'], p1b['alert']))\n"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "name": "python",
      "version": "3.11"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 5
}
